In [5]:
from confluent_kafka.admin import AdminClient, NewTopic, NewPartitions
import logging

logging.basicConfig()

a = AdminClient({'bootstrap.servers': 'localhost:9092,localhost:9093,localhost:9094'})


In [6]:
def create_topics(a, topics):
    """ Create topics """

    new_topics = [NewTopic(topic, num_partitions=1, replication_factor=2) for topic in topics]
    # Call create_topics to asynchronously create topics, a dict
    # of <topic,future> is returned.
    fs = a.create_topics(new_topics)

    # Wait for operation to finish.
    # Timeouts are preferably controlled by passing request_timeout=15.0
    # to the create_topics() call.
    # All futures will finish at the same time.
    for topic, f in fs.items():
        try:
            f.result()  # The result itself is None
            print("Topic {} created".format(topic))
        except Exception as e:
            print("Failed to create topic {}: {}".format(topic, e))

In [7]:
topic_names = ["trade","processing_output"]

create_topics(a,topics=topic_names)

Failed to create topic trade: KafkaError{code=TOPIC_ALREADY_EXISTS,val=36,str="Topic 'trade' already exists."}
Topic processing_output created


In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode
from pyspark.sql.functions import split

In [3]:
spark = SparkSession \
	.builder \
	.appName("StructuredNetworkWordCount") \
	.getOrCreate()

In [8]:
df = spark \
  .readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "localhost:9092,localhost:9093,localhost:9094") \
  .option("subscribe", "trade") \
  .load()

AnalysisException: 'Failed to find data source: kafka. Please deploy the application as per the deployment section of "Structured Streaming + Kafka Integration Guide".;'

In [ ]:
words = lines.select(
	explode(
    	split(lines.value, ' ')
	).alias('word')
)

In [ ]:
# Generate running word count
wordCounts = words.groupBy('word').count()

In [ ]:
query = df \
  .writeStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "localhost:9092,localhost:9093,localhost:9094") \
  .option("topic", "processing_output") \
  .start()

In [ ]:
query.awaitTermination()